In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
from pathlib import Path

In [2]:
shakespeare_url = "https://homl.info/shakespeare"
filepath = tf.keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open (filepath) as f:
    shakespeare_text = f.read()
print(shakespeare_text[:80])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.


In [3]:
# we will use character level encoding rather than word level encoding
text_vec_layer = tf.keras.layers.TextVectorization(split="character", standardize="lower")
text_vec_layer.adapt([shakespeare_text])
encoded = text_vec_layer([shakespeare_text])[0]

In [4]:
# we subtract 2 from the character IDs to get the number of distinct characters and total number of characters
encoded -= 2
n_tokens = text_vec_layer.vocabulary_size() - 2
dataset_size = len(encoded)

In [5]:
# we can next turn this into a dataset of consecutive windows
def to_dataset(sequence, length, shuffle=False, seed=None, batch_size=32):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda window_ds: window_ds.batch(length + 1))
    if shuffle:
        ds = ds.shuffle(100_000, seed=seed)
    ds = ds.batch(batch_size)
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

In [6]:
length = 100
tf.random.set_seed(42)
train_set = to_dataset(encoded[:1_000_000], length=length, shuffle=True, seed=42)
valid_set = to_dataset(encoded[1_000_000:1_060_000], length=length)
test_set = to_dataset(encoded[1_060_000:], length=length)

In [7]:
url = "https://github.com/ageron/data/raw/main/shakespeare_model.tgz"
path = tf.keras.utils.get_file("shakespeare_model.tgz", url, extract=True)
model_path = Path(path).with_name("shakespeare_model")
shakespeare_model = tf.keras.models.load_model(model_path)

In [8]:
y_proba = shakespeare_model.predict(["To be or not to b"])[0, -1]
y_pred = tf.argmax(y_proba)  # choose the most probable character ID
text_vec_layer.get_vocabulary()[y_pred + 2]

1/1 [==============================] - 1s 920ms/step


'e'

In [10]:
# to keep the model from repeating itself, we can sample the next character randomly, with a probability equal to the probability given by the model
log_probabs = tf.math.log([[.5, .4, .1]])
tf.random.set_seed(42)
tf.random.categorical(log_probabs, num_samples=8)

<tf.Tensor: shape=(1, 8), dtype=int64, numpy=array([[0, 1, 0, 2, 1, 0, 0, 1]])>

In [14]:
# to have more control we can divide the logits by a temperature parameter close to 0 favors high probability characters, while a high temperature favors all characters equally
def next_char(text, temperature=1):
    y_proba = shakespeare_model.predict([text])[0, -1:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1)[0, 0]
    return text_vec_layer.get_vocabulary()[char_id + 2]

# we can next write a function to repeatedly call this function
def extend_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [15]:
# we can now generate some text
tf.random.set_seed(42)
print(extend_text("To be or not to be", temperature=0.01))

1/1 [==============================] - 0s 39ms/step
To be or not to be the duke
as it is a proper strange death,
and the


In [16]:
print(extend_text("To be or not to be", temperature=1))

1/1 [==============================] - 0s 43ms/step
To be or not to behold?

second push:
gremio, lord all, a sistermen,


In [20]:
print(extend_text("To be or not to be", temperature=2))

1/1 [==============================] - 0s 42ms/step
To be or not to be it:
lest weder sit, font, abhord lesix
us in this


In [21]:
# for a stateful rnn we need to make sure to use sequential nonoverlapping windows
# batching is also more complicated since we need to make sure that the first batch ends at the end of a sequence and the next batch starts at the beginning of a sequence
# we can create a helper function to create a dataset of windows of text
def to_dataset_for_stateful_rnn(sequence, length):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift=length, drop_remainder=True)
    ds = ds.flat_map(lambda window: window.batch(length + 1)).batch(1)
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)


stateful_train_set = to_dataset_for_stateful_rnn(encoded[:1_000_000], length)
stateful_valid_set = to_dataset_for_stateful_rnn(encoded[1_000_000:1_060_000], length)
stateful_test_set = to_dataset_for_stateful_rnn(encoded[1_060_000:], length)
# not going to retrain the model since it takes a long time